In [43]:
import torch
import torch.nn as nn
import resnet
import model

maps = 32
dim_feature = 7 * 7
basenet = resnet.resnet18(True, maps=maps)
# lstm = nn.LSTM(dim_feature, dim_feature, 2, bidirectional=True, batch_first=True)
lstm = nn.LSTM(maps * dim_feature, maps * dim_feature, 3, bidirectional=False, batch_first=True)
linear_after_lstm = nn.Linear(maps * dim_feature, maps * dim_feature)

input = torch.randn(4,3,224,224)
feature = basenet(input)
print(feature.size())

net = model.Model()
gaze = net(input)
print(gaze)


torch.Size([4, 32, 7, 7])
tensor([ 0.6539, -0.0660], grad_fn=<AddBackward0>)


In [35]:
batch_size = feature.size(0)
feature = feature.flatten(1)

feature = linear_after_lstm(feature)

print(batch_size)
print(feature.size())

4
torch.Size([4, 1568])


In [36]:
lstm_feature, _ = lstm(feature)
print(len(_))
print(_)
print(lstm_feature.size())

print(lstm_feature[-1, :])
# lstm_feature = lstm_feature[-1, :]

2
(tensor([[ 0.0955,  0.1166,  0.0040,  ..., -0.1269, -0.0136,  0.0671],
        [-0.0006,  0.0019,  0.0401,  ...,  0.0326,  0.0098, -0.0415],
        [ 0.0125, -0.0064, -0.0055,  ..., -0.0109, -0.0066,  0.0035]],
       grad_fn=<SqueezeBackward1>), tensor([[ 0.1589,  0.2020,  0.0085,  ..., -0.3171, -0.0346,  0.1227],
        [-0.0013,  0.0036,  0.0809,  ...,  0.0665,  0.0203, -0.0848],
        [ 0.0251, -0.0127, -0.0112,  ..., -0.0218, -0.0129,  0.0071]],
       grad_fn=<SqueezeBackward1>))
torch.Size([4, 1568])
tensor([ 0.0125, -0.0064, -0.0055,  ..., -0.0109, -0.0066,  0.0035],
       grad_fn=<SliceBackward0>)


In [37]:
d_model = maps * dim_feature
nhead = 8
dropout = 0.1
self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)

q=k=lstm_feature
src2, weight = self_attn(q,k,lstm_feature)
print(src2.size())
print(weight.size())
print(src2)
print(weight)
print(weight[0])


torch.Size([4, 1568])
torch.Size([4, 4])
tensor([[ 0.0033, -0.0007,  0.0032,  ...,  0.0019, -0.0002,  0.0010],
        [ 0.0051, -0.0013,  0.0043,  ...,  0.0022,  0.0016, -0.0002],
        [ 0.0042, -0.0016,  0.0045,  ...,  0.0015,  0.0013, -0.0003],
        [ 0.0042, -0.0033,  0.0038,  ...,  0.0019,  0.0016, -0.0005]],
       grad_fn=<SqueezeBackward1>)
tensor([[0.2431, 0.1736, 0.2431, 0.2431],
        [0.2778, 0.2778, 0.2778, 0.2431],
        [0.2083, 0.2431, 0.2778, 0.2431],
        [0.2431, 0.2778, 0.2083, 0.2431]], grad_fn=<SqueezeBackward1>)
tensor([0.2431, 0.1736, 0.2431, 0.2431], grad_fn=<SelectBackward0>)


In [38]:
cls_token = nn.Parameter(torch.randn(1, 1, 32))
cls = cls_token.repeat((1, 3, 2))
print(cls.size())
print(cls)



# tr_feature = feature.permute(2, 0, 1)

# print(tr_feature.size())

torch.Size([1, 3, 64])
tensor([[[ 0.2444,  0.2796,  0.1718,  0.6965,  0.1327,  0.3503, -0.8870,
          -1.5509, -0.5613, -1.5994, -0.3639,  0.7586, -1.0621, -0.4890,
           0.1990, -0.1196,  0.8081,  0.9146,  0.7055,  1.4039,  0.6116,
           0.4986, -0.2072, -0.4875,  0.5237,  0.7952, -1.7667,  0.1355,
           1.6989,  0.0913, -0.3792, -0.8600,  0.2444,  0.2796,  0.1718,
           0.6965,  0.1327,  0.3503, -0.8870, -1.5509, -0.5613, -1.5994,
          -0.3639,  0.7586, -1.0621, -0.4890,  0.1990, -0.1196,  0.8081,
           0.9146,  0.7055,  1.4039,  0.6116,  0.4986, -0.2072, -0.4875,
           0.5237,  0.7952, -1.7667,  0.1355,  1.6989,  0.0913, -0.3792,
          -0.8600],
         [ 0.2444,  0.2796,  0.1718,  0.6965,  0.1327,  0.3503, -0.8870,
          -1.5509, -0.5613, -1.5994, -0.3639,  0.7586, -1.0621, -0.4890,
           0.1990, -0.1196,  0.8081,  0.9146,  0.7055,  1.4039,  0.6116,
           0.4986, -0.2072, -0.4875,  0.5237,  0.7952, -1.7667,  0.1355,
        

In [39]:
x = nn.Parameter(torch.randn(2,3))
print(x)
print(torch.cat((x,x,x),0))
print(torch.cat((x,x,x),1))

Parameter containing:
tensor([[-0.3730,  1.4797,  0.6200],
        [ 0.2864,  1.4627, -1.5407]], requires_grad=True)
tensor([[-0.3730,  1.4797,  0.6200],
        [ 0.2864,  1.4627, -1.5407],
        [-0.3730,  1.4797,  0.6200],
        [ 0.2864,  1.4627, -1.5407],
        [-0.3730,  1.4797,  0.6200],
        [ 0.2864,  1.4627, -1.5407]], grad_fn=<CatBackward0>)
tensor([[-0.3730,  1.4797,  0.6200, -0.3730,  1.4797,  0.6200, -0.3730,  1.4797,
          0.6200],
        [ 0.2864,  1.4627, -1.5407,  0.2864,  1.4627, -1.5407,  0.2864,  1.4627,
         -1.5407]], grad_fn=<CatBackward0>)


In [40]:
encoder_layer = model.TransformerEncoderLayer(maps*4, nhead, 512, dropout)

encoder_norm = nn.LayerNorm(maps*4)
# num_encoder_layer: deeps of layers

encoder = model.TransformerEncoder(encoder_layer, 6, encoder_norm)

In [41]:
from easydict import EasyDict as edict
import yaml
import ctools
import importlib
import os
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter


config = "./config/config_gaze360.yaml"

person = None

config = edict(yaml.load(open(config), Loader=yaml.FullLoader))

config = config.train

config.person = person

config.device = 1

print("=====================>> (Begin) Training params << =======================")

print(ctools.DictDumps(config))

print("=====================>> (End) Training params << =======================")

dataloader = importlib.import_module("reader." + config.reader)
device = torch.device(config.device)
torch.cuda.set_device(device)
cudnn.benchmark = True

data = config.data
save = config.save
params = config.params
average_loss_num = config.ave_loss

writer = SummaryWriter()

print("========================> Read Data <========================")
if config.person is not None:
    if data.isFolder:
        data, folder = ctools.readfolder(data, [config.person], reverse=True)

    save_name = folder[config.person]

    dataset = dataloader.loader(data, params.batch_size, shuffle=True, num_workers=6)

    save_path = os.path.join(save.metapath, save.folder, f"checkpoint/{save_name}")

else:
    if data.isFolder:
        data, _ = ctools.readfolder(data)

    dataset = dataloader.loader(data, params.batch_size, shuffle=True, num_workers=8)

    save_path = os.path.join(save.metapath, save.folder, "checkpoint")


if not os.path.exists(save_path):
    os.makedirs(save_path)


=====================>> (Begin) Training params << =======================
{
    "params": {
        "batch_size": 4,
        "epoch": 40,
        "lr": 0.0005,
        "decay": 0.5,
        "decay_step": 3,
        "warmup": 0
    },
    "save": {
        "metapath": "/home/zjx/Gaze/Output/HybirdGaze",
        "folder": "gaze360",
        "model_name": "trans6",
        "step": 5
    },
    "data": {
        "image": "/home/zjx/Gaze/Datasets/gaze360_processed/Image",
        "label": "/home/zjx/Gaze/Datasets/gaze360_processed/Label/train.label",
        "header": true,
        "name": "gaze360",
        "isFolder": false
    },
    "pretrain": {
        "enable": false,
        "path": "PLACEHOLDER",
        "device": "PLACEHOLDER"
    },
    "device": 1,
    "ave_loss": 1000,
    "reader": "reader",
    "person": null
}
=====================>> (End) Training params << =======================
========================> Read Data <========================
-- [Read Data]: Source: /home/z

In [42]:
for i, (data,label) in enumerate(dataset):
    print(data.face.size())
    print(data)
    print(label)
    break

torch.Size([4, 3, 224, 224])
{'face': tensor([[[[0.2706, 0.2706, 0.2745,  ..., 0.1255, 0.1216, 0.1216],
          [0.2706, 0.2706, 0.2745,  ..., 0.1255, 0.1216, 0.1216],
          [0.2863, 0.2863, 0.2784,  ..., 0.1137, 0.1098, 0.1098],
          ...,
          [0.1412, 0.1412, 0.1490,  ..., 0.2392, 0.2471, 0.2510],
          [0.1373, 0.1412, 0.1490,  ..., 0.2353, 0.2471, 0.2549],
          [0.1333, 0.1373, 0.1490,  ..., 0.2353, 0.2471, 0.2549]],

         [[0.3333, 0.3333, 0.3294,  ..., 0.1412, 0.1333, 0.1333],
          [0.3333, 0.3333, 0.3294,  ..., 0.1412, 0.1333, 0.1333],
          [0.3412, 0.3412, 0.3333,  ..., 0.1294, 0.1216, 0.1216],
          ...,
          [0.1333, 0.1333, 0.1333,  ..., 0.2824, 0.2902, 0.2941],
          [0.1294, 0.1333, 0.1333,  ..., 0.2784, 0.2902, 0.2980],
          [0.1255, 0.1294, 0.1333,  ..., 0.2784, 0.2902, 0.2980]],

         [[0.3804, 0.3804, 0.3765,  ..., 0.1608, 0.1529, 0.1529],
          [0.3804, 0.3804, 0.3765,  ..., 0.1608, 0.1529, 0.1529],
    

In [49]:
gaze = torch.randn(2)
print(gaze.size())
gaze.reshape(2,1)
print(gaze.size())

torch.Size([2])
torch.Size([2])
